In [1]:
import gzip
import zipfile
import json
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy as np
import pandas as pd
import string
import random
import string
from sklearn import linear_model
path = "C://niha//cse158//data"

In [6]:
import json
 
filter_path = path+'//filter_all_t.json'
 
# Open and read the entire content of the JSON file
with open(filter_path, 'r') as json_file:
    filter_data_single_line = json_file.read()
 
# Parse the JSON data
try:
    json_filter_data = json.loads(filter_data_single_line)
    # Now you can work with the parsed JSON data
    print(json_filter_data)
except json.JSONDecodeError as err:
    print(f'Error decoding JSON: {err}')
    

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [11]:
imagerev_path = path+'//image_review_all.json'

with open(imagerev_path, 'r') as json_file:
    for line_number, line in enumerate(json_file,1):
        try:
            imagerev_data = json.loads(line)
            print(imagerev_data)
        except json.JSONDecodeError as err:
            print(f'Error decoding JSON: {err}')

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [12]:
json_filter_data['train'][0]

{'business_id': '60567465d335d0abfb415b26',
 'user_id': '101074926318992653684',
 'rating': 4,
 'review_text': 'The tang of the tomato sauce is outstanding. And the crust is a meal, as it should be. Order a whole pie fresh.',
 'pics': ['AF1QipM-2IRmvitARbcJr7deWfe5hyVBg_ArPMQSYvq0',
  'AF1QipPWhe1OP80YPU40J6-XIdxbJIe57vKm8TTjve31',
  'AF1QipNuKWM65S9ZFQykvdIhKUliE6K1VBxssTUYyl8d',
  'AF1QipOJng1JS_1hmpfhAVrr7hE89dcoOtdy-Z6cOO9x'],
 'history_reviews': [['101074926318992653684_6056272797d555cc6fb0d147',
   'The pizza here is the real deal, perfect in every way except for the crust, which was more cracker crisp than chewy bread. Also recommend the meatball parmigiana hero, beefy, fluffy, overflowing, tangy, chunky tomato sauce, toothsome bread.'],
  ['101074926318992653684_604a65c2c6dc737bce7e5a3d',
   'Omg the tomato sauce is everything, in the meatball appetizer, pizza & as a dip for that scratch-made focaccia. Farm-fresh salad. A variety of artisan toppings. Ask for the chili oil! Goes

In [13]:
imagerev_data

{'business_id': '60401a5466e8bd764351f091',
 'user_id': '116057206044139721905',
 'rating': 5,
 'review_text': '(Translated by Google) Excellent food, fair price\n\n(Original)\nExcelente food, fair price',
 'pics': [{'id': 'AF1QipMLnu-Q6FY0LCiHAJIhDQfgtJGYsqt1Rk2_cAW5',
   'url': ['https://lh5.googleusercontent.com/p/AF1QipMLnu-Q6FY0LCiHAJIhDQfgtJGYsqt1Rk2_cAW5=w150-h150-k-no-p']}]}

In [14]:
json_data.keys()

NameError: name 'json_data' is not defined

In [ ]:
[len(json_data[i]) for i in json_data.keys()]